### Algoritmo de Clustering: K-Means

**K-Means** es un *algoritmo de aprendizaje no-supervisado* utilizado para resolver problemas de *clustering*. Su objetivo es particionar un conjunto de datos en $K$ grupos, llamados *clusters*, donde cada plato pertenece al cluster cuyo *centroide* (el punto medio del cluster) es más cercano. El algoritmo funciona iterativamente para asignar cada punto de datos a uno de los $K$ clusters basándose en las características que los hacen similares, minimizando la varianza intra-cluster.

--- 

Dado un conjunto de datos $X = \{ \mathbf{x}_{i} \}_{i=1}^n$ donde cada $\mathbf{x}_{i} \in \mathbb{R}^d$, y un número entero $k$ que representa la cantidad de clusters a formar, el objetivo de K-Means es encontrar una partición $C = \{ C_{j} \}_{j = 1}^k$ que minimice una métrica como **Función de Distorsión**: 
$$J(C, \mu) = \sum_{i=1}^{n} \|x^{(i)} - \mu_{C^{(i)}}\|^2$$
donde:
- $C_{j}:$ $j$-ésimo cluster
- $\mu_{j}:$ centroide (vector de medias) del cluster $C_{j}$
- $|| \mathbf{x} - \mu_{j} ||:$ representa la **Distancia Euclidiana** entre el punto $\mathbf{x}$ y el centroide $\mu_{j}$

La Función de Distorsión $J$ mide la suma de las distancias al cuadrado entre cada ejemplo de entrenamiento $x^{(i)}$ y el centroide de cluster $\mu_{C^{(i)}}$ al que ha sido asignado. 

---

La función $J$ es una función no convexa, por lo que el descenso coordenado sobre $J$ no está garantizado que converja al mínimo global. En otras palabras, K-Means puede ser susceptible a óptimos locales. Muy a menudo K-Means funcionará bien y producirá agrupamientos muy buenos a pesar de esto. 

Una alternativa y una práctica común es ejecutar K-Means muchas veces y usando diferentes valores iniciales aleatorios para los centroides de cluster ($\mu_j$). Luego, de entre todos los diferentes agrupamientos encontrados, elegir el que da la distorsión más baja $J(C, \mu)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from IPython.display import display
from sklearn import metrics 
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans 

import warnings
warnings.filterwarnings('ignore')

#### Generación del Dataset y Visualización

In [ ]:
X, y = make_blobs(
  n_samples=5000, 
  n_features=2, 
  centers=3, 
  random_state=42
)

In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', alpha=0.6, edgecolors='w', linewidth=0.5)
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#### Aplicación de K-Means

In [ ]:
kmeans = KMeans(
  n_clusters=3,
  n_init=3,
  init='random',
  tol=1e-4, 
  random_state=42,
  verbose=False
)
kmeans.fit(X)

**Resultados del Entrenamiento**:
- `labels_`: vector con los clusters a los cuales pertenece cada instancia de entrenamiento
- `cluster_centers_`: matriz con los centroides de cada grupo

In [ ]:
y_kmeans = kmeans.predict(X) 
centroids = kmeans.cluster_centers_
display(centroids)

In [ ]:
plt.figure(figsize=(10, 7))
# Datos
plt.scatter(
  X[:, 0], X[:, 1], 
  c=kmeans.labels_, 
  cmap='viridis', 
  alpha=0.6, 
  s=40,
  edgecolors='black',
  linewidth=0.5
)
# Centroides
plt.scatter(
  kmeans.cluster_centers_[:, 0], 
  kmeans.cluster_centers_[:, 1], 
  c='red', 
  marker='X', 
  s=300,
  linewidths=3,
  edgecolors='black',
  label='Centroides'
)
plt.title(f'Clustering K-Means (k={kmeans.n_clusters})', fontsize=14, fontweight='bold')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.colorbar(label='Cluster')

plt.tight_layout()
plt.show()

#### Métricas de Evaluación
Dado que se tienen métricas reales, se pueden calcular algunas métricas: `silhouette_score` y `adjusted_rand_score`.

In [ ]:
from sklearn.metrics import silhouette_score, adjusted_rand_score

# Silhouette Score: mide la calidad de los clusters, sin necesidad de etiquetas reales
sil_score = silhouette_score(X, y_kmeans)
print(f"Silhouette Score: {sil_score:.3f}")

# Adjusted Rand Index: compara con las etiquetas reales
ari = adjusted_rand_score(y, y_kmeans)
print(f"Adjusted Rand Index: {ari:.3f}")

#### Ejemplo de K-Means con Diferentes Parámetros 

In [ ]:
kmeans_k4 = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans_k4.fit(X)
y_kmeans_k4 = kmeans_k4.predict(X)

# Visualización
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans_k4, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
plt.scatter(kmeans_k4.cluster_centers_[:, 0], kmeans_k4.cluster_centers_[:, 1], 
            c='red', marker='X', s=200, label='Centroides (k=4)')
plt.title('K-Means con k=4')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster predicho')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

# Calcular inercia (suma de distancias al cuadrado)
print(f"Inercia (k=3): {kmeans.inertia_:.2f}")
print(f"Inercia (k=4): {kmeans_k4.inertia_:.2f}")

#### Solución para Elegir K Óptimo
**Elbow Method**: Técnica heurística utilizada para determinar el número óptimo de clusters ($k$) en algoritmos de clustering como K-Means. Este método se basa en la idea de que a medida que se aumenta el número de clusters ($k$), la inercia (función de distorsión) disminuye. 
- Cuando $k$ es muy pequeño, cada vez que aumentamos $k$, la inercia disminuye significativamente
- Cuando $k$ se acerca al número óptimo real de clusters, la mejora (reducción de inercia) se vuelve menos pronunciada

In [ ]:
inertias = []
K = range(1, 11)
for k in K:
  kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)
  kmeans_model.fit(X)
  inertias.append(kmeans_model.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K, inertias, 'bo-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método para Determinar K Óptimo')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

### K-Means desde Cero (K-Means from Scratch)

In [ ]:
class KMeansFromScratch:
  """Implementación del algoritmo K-Means desde cero.

  Parámetros:
  -----------
  n_clusters : int, default=3
    Número de clusters a formar.
  max_iters : int, default=100
    Número máximo de iteraciones.
  tol : float, default=1e-4
    Tolerancia para declarar convergencia (cambio en inercia).
  random_state : int, default=None
    Semilla para reproducibilidad.
  init_method : str, default='random'
    Método de inicialización: 'random' o 'k-means++'.
  """

  def __init__(self, n_clusters=3, max_iters=100, tol=1e-4,
               random_state=None, init_method='random'):
    self.n_clusters = n_clusters
    self.max_iters = max_iters
    self.tol = tol
    self.random_state = random_state
    self.init_method = init_method
    self.centroids = None
    self.labels = None
    self.inertia = None
    self.history = []  # Para guardar historial de inercia

  def _initialize_centroids(self, X):
    "Inicializa los centroides usando el método especificado"
    np.random.seed(self.random_state)
    n_samples = X.shape[0]

    if self.init_method == 'random':
      # Seleccionar k puntos aleatorios del dataset
      indices = np.random.choice(n_samples, self.n_clusters, replace=False)
      centroids = X[indices]

    elif self.init_method == 'k-means++':
      # Método k-means++ para una mejor inicialización
      centroids = np.zeros((self.n_clusters, X.shape[1]))

      # Paso 1: Elegir el primer centroide aleatoriamente
      first_idx = np.random.randint(n_samples)
      centroids[0] = X[first_idx]

      # Paso 2: Para cada centroide restante
      for i in range(1, self.n_clusters):
        # Calcular distancias mínimas al centroide más cercano
        distances = np.zeros(n_samples)
        for j in range(n_samples):
          # Distancia del punto j a todos los centroides existentes
          dist_to_centroids = np.linalg.norm(X[j] - centroids[:i], axis=1)
          distances[j] = np.min(dist_to_centroids)**2

        # Probabilidad proporcional al cuadrado de la distancia
        probabilities = distances / distances.sum()

        # Elegir nuevo centroide basado en las probabilidades
        new_idx = np.random.choice(n_samples, p=probabilities)
        centroids[i] = X[new_idx]

    return centroids

  def _compute_distances(self, X, centroids):
    """Calcula la distancia entre cada punto y cada centroide

    Args:
      X: (n_samples, n_features)
      centroids: (n_clusters, n_features)

    Returns: (n_samples, n_clusters)
    """
    distances = np.zeros((X.shape[0], self.n_clusters))
    for i, centroid in enumerate(centroids):
      # Distancia euclidiana al cuadrado
      distances[:, i] = np.sum((X - centroid)**2, axis=1)

    return distances

  def _assign_clusters(self, X):
    "Asigna cada punto al cluster más cercano"
    distances = self._compute_distances(X, self.centroids)
    # Asignar al cluster con distancia mínima
    return np.argmin(distances, axis=1)

  def _update_centroids(self, X, labels):
    "Actualiza los centroides como la media de los puntos en cada cluster"
    new_centroids = np.zeros((self.n_clusters, X.shape[1]))

    for i in range(self.n_clusters):
      # Filtrar puntos que pertenecen al cluster i
      cluster_points = X[labels == i]

      if len(cluster_points) > 0:
        new_centroids[i] = np.mean(cluster_points, axis=0)
      else:
        # Si un cluster queda vacío, reinicializar su centroide aleatoriamente
        new_centroids[i] = X[np.random.randint(X.shape[0])]

    return new_centroids

  def _compute_inertia(self, X, labels):
    "Calcula la inercia (suma de distancias al cuadrado a los centroides)"
    inertia = 0.0
    for i in range(self.n_clusters):
      cluster_points = X[labels == i]
      if len(cluster_points) > 0:
        distances = np.sum((cluster_points - self.centroids[i])**2)
        inertia += distances
    return inertia

  def fit(self, X):
    """Entrena el modelo K-Means con los datos X.

    Parámetros:
    -----------
    X : array-like, shape (n_samples, n_features)
      Datos de entrenamiento.

    Retorna:
    --------
    self : Modelo entrenado.
    """
    # Verificar que X es un array numpy
    X = np.array(X)

    # Inicializar centroides
    self.centroids = self._initialize_centroids(X)

    # Bucle principal del algoritmo
    for iteration in range(self.max_iters):
      # Paso 1: Asignar clusters
      self.labels = self._assign_clusters(X)

      # Paso 2: Actualizar centroides
      new_centroids = self._update_centroids(X, self.labels)

      # Paso 3: Calcular inercia
      self.inertia = self._compute_inertia(X, self.labels)
      self.history.append(self.inertia)

      # Paso 4: Verificar convergencia
      centroids_change = np.linalg.norm(new_centroids - self.centroids)

      if centroids_change < self.tol:
        print(f"Convergió en la iteración {iteration + 1}")
        break

      # Actualizar centroides para la siguiente iteración
      self.centroids = new_centroids

      # Mostrar progreso cada 10 iteraciones
      if (iteration + 1) % 10 == 0:
        print(f"Iteración {iteration + 1}, Inercia: {self.inertia:.4f}")

    print(f"Algoritmo finalizado. Inercia final: {self.inertia:.4f}")
    return self

  def predict(self, X):
    """Predice los clusters para nuevos datos.

    Parámetros:
    -----------
    X : array-like, shape (n_samples, n_features)
      Nuevos datos.

    Retorna:
    --------
    labels : array, shape (n_samples,)
      Etiquetas de cluster para cada punto.
    """
    X = np.array(X)
    return self._assign_clusters(X)

  def fit_predict(self, X):
    "Equivalente a fit() seguido de predict()."
    self.fit(X)
    return self.labels

  def get_params(self):
    "Retorna los parámetros del modelo."
    return {
      'n_clusters': self.n_clusters,
      'max_iters': self.max_iters,
      'tol': self.tol,
      'random_state': self.random_state,
      'init_method': self.init_method
    }

  def score(self, X):
    "Retorna la inercia negativa (para compatibilidad con GridSearch)."
    if self.centroids is None:
      raise ValueError("El modelo debe ser entrenado primero.")
    return -self._compute_inertia(X, self.predict(X))

In [ ]:
# Crear y entrenar el modelo
kmeans_custom = KMeansFromScratch(
  n_clusters=3,
  max_iters=100,
  tol=1e-4,
  random_state=42,
  init_method='k-means++'
)

# Entrenar el modelo
labels_custom = kmeans_custom.fit_predict(X)

# Visualizar resultados
plt.figure(figsize=(15, 5))

# Subplot 1: Datos originales
plt.subplot(1, 3, 1)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=50, alpha=0.7)
plt.title('Datos Originales con Clusters Reales')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster real')

# Subplot 2: Resultados de K-Means personalizado
plt.subplot(1, 3, 2)
plt.scatter(X[:, 0], X[:, 1], c=labels_custom, cmap='viridis', s=50, alpha=0.7)
plt.scatter(kmeans_custom.centroids[:, 0], kmeans_custom.centroids[:, 1], 
            marker='X', s=200, c='red', label='Centroides')
plt.title('K-Means Personalizado')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster predicho')
plt.legend()

# Subplot 3: Evolución de la inercia
plt.subplot(1, 3, 3)
plt.plot(range(1, len(kmeans_custom.history) + 1), kmeans_custom.history, 'bo-')
plt.xlabel('Iteración')
plt.ylabel('Inercia')
plt.title('Evolución de la Inercia')
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Mostrar métricas
print(f"Número de iteraciones realizadas: {len(kmeans_custom.history)}")
print(f"Inercia final: {kmeans_custom.inertia:.4f}")
print(f"Centroides finales:\n{kmeans_custom.centroids}")

In [ ]:
# Entrenar K-Means de scikit-learn para comparar
kmeans_sklearn = KMeans(n_clusters=3, random_state=42, init='k-means++')
labels_sklearn = kmeans_sklearn.fit_predict(X)

# Comparar resultados
print("="*50)
print("COMPARACIÓN CON SCIKIT-LEARN")
print("="*50)

print(f"\nCentroides (Personalizado):\n{kmeans_custom.centroids}")
print(f"\nCentroides (Scikit-learn):\n{kmeans_sklearn.cluster_centers_}")

print(f"\nInercia (Personalizado): {kmeans_custom.inertia:.4f}")
print(f"Inercia (Scikit-learn): {kmeans_sklearn.inertia_:.4f}")

ari = adjusted_rand_score(labels_sklearn, labels_custom)
print(f"\nAdjusted Rand Score (similitud): {ari:.4f}")

# Visualizar comparación
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=labels_custom, cmap='viridis', s=50, alpha=0.7)
plt.scatter(kmeans_custom.centroids[:, 0], kmeans_custom.centroids[:, 1], 
            marker='X', s=200, c='red', label='Centroides')
plt.title('K-Means Personalizado')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=labels_sklearn, cmap='viridis', s=50, alpha=0.7)
plt.scatter(kmeans_sklearn.cluster_centers_[:, 0], kmeans_sklearn.cluster_centers_[:, 1], 
            marker='X', s=200, c='red', label='Centroides')
plt.title('K-Means Scikit-learn')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

plt.tight_layout()
plt.show()

In [ ]:
# Probar con diferentes números de clusters
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
k_values = [2, 3, 4, 5, 6, 7]

for idx, k in enumerate(k_values):
  ax = axes[idx // 3, idx % 3]

  # Entrenar modelo
  kmeans_test = KMeansFromScratch(
      n_clusters=k,
      max_iters=100,
      random_state=42,
      init_method='k-means++'
  )
  labels_test = kmeans_test.fit_predict(X)

  # Visualizar
  ax.scatter(X[:, 0], X[:, 1], c=labels_test, cmap='viridis', s=30, alpha=0.6)
  ax.scatter(kmeans_test.centroids[:, 0], kmeans_test.centroids[:, 1],
             marker='X', s=150, c='red', label=f'k={k}')
  ax.set_title(f'K-Means con k={k}\nInercia: {kmeans_test.inertia:.2f}')
  ax.set_xlabel('Feature 1')
  ax.set_ylabel('Feature 2')
  ax.legend()

plt.tight_layout()
plt.show()